In [1]:
import csv
import re
import time
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException

In [2]:
csv_cols = {
    "NIT/RFP NO": "ref_no",
    "Name of Work / Subwork / Packages": "title",
    "Estimated Cost": "tender_value",
    "Bid Submission Closing Date & Time": "bid_submission_end_date",
    "EMD Amount": "emd",
    "Bid Opening Date & Time": "bid_open_date"
}

In [3]:
service = Service(executable_path="./chromedriver.exe")
driver = Chrome(service=service)
def get_js_text(element):
    return driver.execute_script("return arguments[0].textContent.trim();", element)
def is_valid_tender_id(text):
    return bool(re.search(r'\d', text))

In [4]:
try:
    driver.get("https://etender.cpwd.gov.in/")
    wait = WebDriverWait(driver, 20)
    view_more_button = wait.until(EC.element_to_be_clickable((By.ID, "viewCurrentall")))
    view_more_button.click()
    print("⏳ Waiting for tenders table to load...")
    time.sleep(5)
    tender_count = 0
    all_tenders = []
    while True:
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table tbody tr")))
        rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")
        print(f"Scraping {len(rows)} rows on current page...")
        for i, row in enumerate(rows):
            try:
                cols = row.find_elements(By.TAG_NAME, "td")
                if len(cols) < 8:
                    continue
                tender_id = get_js_text(cols[0])
                if not is_valid_tender_id(tender_id):
                    continue
                tender_count += 1
                nit_rfp_no = get_js_text(cols[1])
                name_of_work = get_js_text(cols[2])
                estimated_cost = get_js_text(cols[4])
                bid_submission = get_js_text(cols[6])
                emd_amount = get_js_text(cols[5])
                bid_opening = get_js_text(cols[7])
                tender_data = {
                    csv_cols["NIT/RFP NO"]: nit_rfp_no,
                    csv_cols["Name of Work / Subwork / Packages"]: name_of_work,
                    csv_cols["Estimated Cost"]: estimated_cost,
                    csv_cols["Bid Submission Closing Date & Time"]: bid_submission,
                    csv_cols["EMD Amount"]: emd_amount,
                    csv_cols["Bid Opening Date & Time"]: bid_opening,
                }

                all_tenders.append(tender_data)

                print(f"{tender_count}. {tender_data['ref_no']}")
            except StaleElementReferenceException:
                print(f" Skipped row {i + 1} due to stale element.")
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "a.paginate_button.next")
            classes = next_button.get_attribute("class")
            if "disabled" in classes:
                print(" Reached last page.")
                break
            else:
                next_button.click()
                print(" Moving to next page...")
                time.sleep(5)
        except Exception as e:
            print(f" Pagination failed: {e}")
            break
    print(f"\n Total tenders scraped: {tender_count}")
    csv_filename = "tenders.csv"
    with open(csv_filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=list(csv_cols.values()))
        writer.writeheader()
        writer.writerows(all_tenders)
    print(f" Data saved to {csv_filename}")

finally:
    driver.quit()


⏳ Waiting for tenders table to load...
Scraping 19 rows on current page...
1. 01/SE/Jalandhar/2025-26
2. 02/AE(E)-IV/DED-203/2025-26/Recall-1
3. 02/CE/NDZ-I/DED-101/2025-26
4. 18//EE(E)/AE(E)/GESD-II/2025-26
5. NIT17/EE/PE/2025-26
6. 04/SE&PD/EE/SURAT/2025-26
7. 18/NIT/EE(E)-Agra/2025-26
8. 06/2025-26/C-Div/Delhi/4C
9. 15/EE(E)(Vizag)/2025-26
10. 11/EE/Q Divn/2025-26
 Moving to next page...
Scraping 19 rows on current page...
11. 05/EE/TRICHY-I/02/NIT/SE(TRICHY)/CPWD/2025-26 - Recall 1
12. 02-R1/NIT/SE&PD/BGPC/2024-25
13. 04/EE/TRICHY-1/2025-26 - Recall 1
14. 13/MyCD/SE/e-tender/2025-26
15. 09/CE/NDZ-3/2025-26
16. 27/2025-26/R-Divn/Delhi
17. 28/2025-26/R-DIVN/DELHI
18. 29/2025-26/R-DIVN/2025-26
19. 30/2025-26/R-DIVN/DELHI
20. 31/2025-26/R-DIVN/DELHI
 Moving to next page...
Scraping 19 rows on current page...
21. 27/EE-E/AE-E/GESD-III/2025-26
22. 94/2024-25/CCED/Chandigarh/Div/4th Call
23. 02/NIT/CE(PATNA)/CPWD/2025-2026
24. 02/NIT/CE/AGT/2025-2026
25. 03/NIT/CE(PATNA)/CPWD/2025-2026
26